<img src="../img/GTK_Logo_Social Icon.jpg" width=175 align="right" />

# Worksheet 2.1:  Exploring Two Dimensional Data -  Answers
This worksheet covers concepts covered in Module 2 - Exploratory Data Analysis in Two Dimensions.  It should take no more than 20-30 minutes to complete.  Please raise your hand if you get stuck.  

There are many ways to accomplish the tasks that you are presented with, however you will find that by using the techniques covered in class, the exercises should be relatively simple. 

## Import the Libraries
For this exercise, we will be using:
* Pandas (http://pandas.pydata.org/pandas-docs/stable/)
* Numpy (https://docs.scipy.org/doc/numpy/reference/)
* Matplotlib (http://matplotlib.org/api/pyplot_api.html)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
plt.style.use('ggplot')
%pylab inline

## Exercise 2.1: Reading various forms of JSON Data
In the `/data/` folder, you will find a series of `.json` files called `dataN.json`, numbered 1-4.  Each file contains the following data:

<table>
    <tr>
        <th></th>
        <th>birthday</th>
        <th>first_name</th>
        <th>last_name</th>
    </tr>
    <tr>
        <td>0</td>
        <td>5\/3\/67</td>
        <td>Robert</td>
        <td>Hernandez</td>
    </tr>
    <tr>
        <td>1</td>
        <td>8\/4\/84</td>
        <td>Steve</td>
        <td>Smith</td>
    </tr>
    <tr>
        <td>2</td>
        <td>9\/13\/91</td>
        <td>Anne</td>
        <td>Raps</td>
    </tr>    
    <tr>
        <td>3</td>
        <td>4\/15\/75</td>
        <td>Alice</td>
        <td>Muller</td>
    </tr>    
</table>

Using the `.read_json()` function and the various configuration options, read all these files into a dataframe.  The documentation is available here: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_json.html.

In [ ]:
DATA_HOME = '../data/'
df1 = pd.read_json(DATA_HOME + 'data1.json')
df1

In [ ]:
df2 = pd.read_json(DATA_HOME + 'data2.json', orient='index')
df2

In [ ]:
df3 = pd.read_json(DATA_HOME + 'data3.json', orient='columns')
df3

In [ ]:
df4 = pd.read_json(DATA_HOME + 'data4.json', orient='split')
df4

## Exercise 2: 
In the data file, there is a webserver file called `hackers-access.httpd`.  For this exercise, you will use this file to answer the following questions:
1.  Which browsers are the top 10 most used browsers in this data?
2.  Which are the top 10 most used operating systems?

In order to accomplish this task, do the following:
1.  Write a function which takes a User Agent string as an argument and returns the relevant data.  HINT:  You might want to use python's `user_agents` module, the documentation for which is available here: (https://pypi.python.org/pypi/user-agents)
2.  Next, apply this function to the column which contains the user agent string.
3.  Store this series as a new column in the dataframe
4.  Count the occurances of each value in the new columns

In [ ]:
import apache_log_parser
from user_agents import parse
#Read in the log file
line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-agent}i\"")

server_log = open(DATA_HOME + "hackers-access.httpd", "r")
parsed_server_data = []
for line in server_log:
    data = {}
    data = line_parser(line)
    parsed_server_data.append( data )

server_df = pd.DataFrame( parsed_server_data  )

In [ ]:
server_df.head()

In [ ]:
#Write the functions
def get_os(x):
    user_agent = parse(x)
    return user_agent.os.family

def get_browser(x):
    user_agent = parse(x)
    return user_agent.browser.family

In [ ]:
#Apply the functions to the dataframe
server_df['os'] = server_df['request_header_user_agent'].apply( get_os )
server_df['browser'] = server_df['request_header_user_agent'].apply( get_browser )

In [ ]:
#Get the top 10 values
server_df['os'].value_counts().head(10)

In [ ]:
server_df['browser'].value_counts().head(10)

# Exercise 3:
Using the `dailybots.csv` film, read the file into a DataFrame and perform the following operations:
1.  Filter the DataFrame to include bots from the Government/Politics Industry.
2.  Calculate the ratio of hosts to orgs and add this as a column to the DataFrame and output the result
3.  Calculate the total number of hosts infected by each BotFam in the Government/Politics Industry.  You should use the `groupby()` function which is documented here: (http://pandas.pydata.org/pandas-docs/stable/groupby.html)


In [ ]:
bots = pd.read_csv(DATA_HOME + 'dailybots.csv')
bots.head()

In [ ]:
gov_bots = bots[['botfam', 'hosts']][bots['industry'] == "Government/Politics"]

In [ ]:
gov_bots['ratio'] = bots['hosts'] / bots['orgs']

In [ ]:
gov_bots.sample(5)

In [ ]:
gov_bots.groupby('botfam', as_index=False).sum()

# Exercise 4:

Read a more ```evil``` JSON ```eve_small.json```, where each line contains a nested JSON object. Derive one DataFrame, where all levels for the ```stats``` key are expanded to a top level column of that DataFrame. Easiest is to natively open the file in Python, loop over each line, use [json.loads](https://docs.python.org/3.5/library/json.html) from the json library, and then [json_normalize](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html) to expand the nested structure to top-level columns, append to a simple Python list and finally call [pd.concat](http://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.concat.html) on the list to get one complete DataFrame.

In [ ]:
def nested_json_to_df(fname_str):
    with open(fname_str, 'r') as f:
        l = []
        for line in f:
            if not line.strip():
                continue
            else:
                d = json.loads(line)
                if 'stats' in d.keys():
                    df_tmp = pd.json_normalize(d)
                    l.append(df_tmp)       
    return pd.concat(l, ignore_index=True)

In [ ]:
eve = nested_json_to_df(DATA_HOME + 'eve_small.json')

In [ ]:
eve.head()

# Exercise 5
In this exercise, you will learn how to convert a PCAP file into JSON and do some basic summarization of the data.  In the `data` directory, you will find a file called `http.pcap`.  Our first step is to convert this to JSON.  To do this we have installed a python module called `pcapview` (docs available here: https://pydigger.com/pypi/pcapview) which can convert the pcap file to JSON.  

Once you've done that, your assignment is to answer the following questions:
1.  What are the most frequent source IP addresses?
2.  How many differnet source ports were accessed?

To do this you will have to load this data into a DataFrame.  Using what we've learned in class, do the following:
1.  Load the data into a DataFrame using the technique of your choice
2.  Extract the requisite columns from the DataFrame, in this case, you want the source IP and source ports
3.  Execute a `value_counts()` on those columns.  

In [ ]:
#Load the data
with open(DATA_HOME + 'http-pcap.json') as data_file:    
    pcap_data = json.load(data_file)

#Normalize it and load it into a DataFrame
df = pd.DataFrame( pd.json_normalize(pcap_data) )

#View the results
df.head()

In [ ]:
df1 = pd.read_json(DATA_HOME + 'http-pcap.json')

In [ ]:
#Extract the source port and count the unique values
df['TCP.sport'].value_counts(dropna=False)

In [ ]:
#Extract the source IP and count the unique values
df['IP.src'].value_counts(dropna=False)